In [28]:
import sqlite3 as sql 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [29]:
db = "plankermira/out/Nordwandschacht.sql"
CAVENAME = "Nordwandschacht"
conn = sql.connect(':memory:')
conn.executescript(open(db).read())

In [30]:
def trig_to_cart(length, bearing, gradient):
    n = np.array([np.cos(np.pi/2 - bearing* np.pi / 180), 
              np.sin(np.pi/2 - bearing* np.pi / 180), 
              np.sin(gradient* np.pi / 180)])
    
    v = length * n / np.linalg.norm(n)
    return v

In [31]:
def returnlrud(stn_id, c):

    # collect LRUD data from a single shot

    c.execute(f"SELECT su.NAME, stn1.NAME, stn2.NAME, sh.LENGTH, sh.BEARING, sh.GRADIENT, sf.FLAG \
            from STATION as stn1, STATION as stn2, SHOT as sh, SURVEY as su \
            LEFT OUTER JOIN SHOT_FLAG sf \
            ON sh.ID == sf.SHOT_ID \
            where stn1.SURVEY_ID == su.ID and stn1.ID == {stn_id} and stn1.ID == sh.FROM_ID and stn2.ID == sh.TO_ID;")

    columns = ["su.NAME", "stn1.NAME", "stn2.NAME", "sh.LENGTH", "sh.BEARING", "sh.GRADIENT", "sf.FLAG" ]


    data = pd.DataFrame(c.fetchall(), columns=columns)

    shots = data[data["stn2.NAME"]!="."]
    splays = data[data["stn2.NAME"]=="."]

    normal_bearing, normal_gradient = np.average(shots[["sh.BEARING", "sh.GRADIENT"]], axis =0)

    n_hat = trig_to_cart(1, normal_bearing, normal_gradient)

    # find the two plane vectors. One is horizontal, 

    u = np.array([n_hat[1], -n_hat[0], 0])
    u_hat = u / np.linalg.norm(u)
    v_hat = np.cross(u_hat, n_hat)

    # project the splays in a plane defined by nhat. 

    splays_proj = []
    for splay in splays.values:
        l, bearing, gradient = splay[[3,4,5]]
        v = trig_to_cart(l, bearing, gradient)

        if np.abs(n_hat @ v / np.linalg.norm(v)) < 0.4:
            spl_proj = np.array([u_hat @ v, v_hat @ v])
            splays_proj.append(spl_proj)

    splays_proj = np.array(splays_proj)

    if splays_proj.shape[0] >= 4:
        
        left = "{:.2f}".format(np.abs(np.min(splays_proj[:,0])))
        right = "{:.2f}".format(np.abs(np.max(splays_proj[:,0])))
        up =  "{:.2f}".format(np.abs(np.max(splays_proj[:,1])))
        down = "{:.2f}".format(np.abs(np.min(splays_proj[:,1])))
    else:
        left, right, up, down = "0.00", "0.00", "0.00", "0.00"

    return left, right, up, down


In [32]:
c = conn.cursor()

c.execute("SELECT ct.TITLE, ct.TOPO_DATE, su2.NAME, su1.NAME, stn1.NAME, su2.NAME, su1.NAME, stn2.NAME, sh.ADJ_LENGTH, sh.BEARING, sh.GRADIENT, sf.FLAG, stn1.ID \
          from STATION as stn1, STATION as STN2, SHOT as sh, SURVEY as su1, SURVEY as su2, CENTRELINE as ct\
          LEFT OUTER JOIN SHOT_FLAG sf \
          ON sh.ID == sf.SHOT_ID \
          where stn1.ID == sh.FROM_ID and stn2.ID == sh.TO_ID and stn2.NAME != '-' and stn2.NAME != '.' \
          and stn1.SURVEY_ID == su1.ID and su1.PARENT_ID == su2.ID and sh.CENTRELINE_ID == ct.ID;")

In [33]:
shot_data = pd.DataFrame(c.fetchall())

In [34]:
shot_data

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Oberer Teil der Tiefenbronner Halle,2023-08-13,nw,1,7,nw,1,0,2.586,24.04,0.8,None,283
1,Oberer Teil der Tiefenbronner Halle,2023-08-13,nw,1,0,nw,1,1,8.994,81.64,-2.7,None,1
2,Oberer Teil der Tiefenbronner Halle,2023-08-13,nw,1,1,nw,1,2,11.743,85.64,28.0,None,41
3,Oberer Teil der Tiefenbronner Halle,2023-08-13,nw,1,2,nw,1,2.1,15.685,11.04,2.5,None,64
4,Oberer Teil der Tiefenbronner Halle,2023-08-13,nw,1,2.1,nw,1,2.2,3.948,19.64,17.2,None,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,Shawshank-Redemption-Schluf,2024-08-14,nw,5,32,nw,5,33,5.126,22.89,-48.7,None,3347
266,Shawshank-Redemption-Schluf,2024-08-14,nw,5,33,nw,5,34,2.968,243.79,-29.3,None,3363
267,Shawshank-Redemption-Schluf,2024-08-14,nw,5,34,nw,5,35,5.438,263.29,-59.4,None,3385
268,Shawshank-Redemption-Schluf,2024-08-14,nw,5,33,nw,5,36,3.923,327.09,8.2,None,3363


In [35]:
_2024_dpl = np.sum(shot_data[np.logical_and(shot_data[3].astype(int)>0, shot_data[11]=="dpl")][8].values)
_2024_total = np.sum(shot_data[shot_data[3].astype(int)>0][8].values)

In [36]:
_2024_total -_2024_dpl
print(_2024_total)
print(_2024_total - _2024_dpl)

2382.77
1808.749


In [37]:
series_titles= shot_data[0].unique()
series_dates = shot_data[1].unique()

In [38]:
topoteams = []
dates = []
for series_date in series_dates:
    topoteam = ""
    sep = " "
    date = sep.join(series_date.split("-")[::-1])
    monthfirst = date[1] + date[0] + date[2]

    c.execute(f"SELECT pp.NAME, pp.SURNAME from PERSON pp, CENTRELINE ct, TOPO tp\
              where tp.PERSON_ID == pp.ID and ct.ID == tp.CENTRELINE_ID and ct.TOPO_DATE == '{series_date}';")
    for name, surname in c.fetchall():
        topoteam+= f"{name} {surname}, "
    
    topoteams.append(topoteam[:-2])
    dates.append(monthfirst)


In [39]:
series_titles

array(['Oberer Teil der Tiefenbronner Halle',
       'Unterer Teil der Tiefenbronner Halle', 'Zugzwang Gang',
       'Eingangschächte der Nordwandschacht', 'Aglarond', 'Vesuv-Dom',
       'Shawshank-Redemption-Schluf'], dtype=object)

In [40]:
duplicate_shot = "#|L#"
survey_name = "SURVEY NAME: {survey_name}"
survey_date = "SURVEY DATE: {survey date}"
survey_team = "SURVEY TEAM: \n{survey team}"
centreline_header = """DECLINATION:    0.00  FORMAT: DMMDLRUDLDAN  CORRECTIONS:  0.00 0.00 0.00

        FROM           TO   LENGTH  BEARING      INC     LEFT       UP     DOWN    RIGHT   FLAGS  COMMENTS"""

In [41]:
feet = 3.2808399

In [42]:
shot_stub = """{: >12}{: >13}{: >9}{: >9}{: >9}{: >9}{: >9}{: >9}{: >9}  {}\n"""

In [43]:
series_titles

array(['Oberer Teil der Tiefenbronner Halle',
       'Unterer Teil der Tiefenbronner Halle', 'Zugzwang Gang',
       'Eingangschächte der Nordwandschacht', 'Aglarond', 'Vesuv-Dom',
       'Shawshank-Redemption-Schluf'], dtype=object)

In [44]:
survey_names = ["NWS1A", "NWS1B", "NWS2", "NWS3", "NWS4A", "NWS4B", "NWS5"]

In [47]:
compass_file = ""

for survey_name, serie, title in zip(survey_names, series_dates, series_titles):

    topoteam = ""
    sep = " "
    date = serie.split("-")[::-1]
    monthfirst = f"{date[1]} {date[0]} {date[2]}"
    

    c.execute(f"SELECT pp.NAME, pp.SURNAME from PERSON pp, CENTRELINE ct, TOPO tp\
              where tp.PERSON_ID == pp.ID and ct.ID == tp.CENTRELINE_ID and ct.TOPO_DATE == '{serie}';")
    
    for name, surname in c.fetchall():
        if f"{name} {surname}" not in topoteam:
            topoteam+= f"{name} {surname}, "

    centreline_stub = """{cavename}
SURVEY NAME: {survey_name}
SURVEY DATE: {formatted_date}   COMMENT: {formatted_name}
SURVEY TEAM:
{formatted_team}
{header}

{shots}
"""
    # extract the shot data  
    ct = shot_data[shot_data[1] == serie]

    formatted_shots = ""

    for index, shot in ct.iterrows():

        stn_from = shot[2] +"." + shot[3] +"." + shot[4]
        stn_to = shot[5] +"." + shot[6] +"." + shot[7]
        tape = f"{float(shot[8]) * feet:.4f}"
        comp = shot[9] 
        inc = shot[10]
        stn1id = shot[12]

        l, r, u, d = returnlrud(stn_id=stn1id, c=c)
        if shot[11] == "dpl":
            flag = duplicate_shot
        else:
            flag = " "

        formatted_shots+= shot_stub.format(stn_from, 
                                           stn_to, 
                                           tape, 
                                           comp, 
                                           inc,
                                           "0.00", 
                                           "0.00", 
                                           "0.00", 
                                           "0.00", 
                                           flag)


    formatted_centreline = centreline_stub.format(cavename =CAVENAME,
                                                  survey_name = survey_name,
                                                  formatted_name = title, 
                                                  formatted_date = monthfirst,
                                                  formatted_team = topoteam[:-2],
                                                  header = centreline_header,
                                                  shots = formatted_shots)
    
    compass_file += formatted_centreline

In [48]:
with open("plankermira/out/Nordwandschacht_conv5.DAT", "w") as f:
    f.write(compass_file)
    f.close()